# Démonstration du système RAG Multimodal

Ce notebook montre comment utiliser le système de Retrieval Augmented Generation (RAG) multimodal qui permet de combiner des requêtes sur des textes et des images grâce aux embeddings CLIP.

## Installation des dépendances

Assurez-vous d'avoir installé toutes les dépendances nécessaires :

In [ ]:
!pip install langchain langchain-core langchain-community langchain-ollama
!pip install torch transformers clip pymupdf pillow chromadb
!pip install matplotlib tqdm numpy requests

## 1. Importer les classes nécessaires

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO

# Importer notre système RAG multimodal
from multimodal_rag import MultimodalEmbedder, MultimodalVectorStore, MultimodalRAG

## 2. Vérifier que Ollama est en cours d'exécution

Avant de commencer, assurez-vous que Ollama est en cours d'exécution et que les modèles nécessaires sont installés.

In [ ]:
import subprocess

# Vérifier si Ollama est en cours d'exécution
try:
    response = requests.get("http://localhost:11434/api/tags")
    if response.status_code == 200:
        print("✅ Ollama est en cours d'exécution")
        # Afficher les modèles disponibles
        models = response.json()["models"]
        print(f"Modèles disponibles: {', '.join([m['name'] for m in models])}")
    else:
        print("❌ Ollama est en cours d'exécution mais a retourné une erreur")
except Exception as e:
    print(f"❌ Ollama n'est pas en cours d'exécution ou n'est pas accessible: {e}")
    print("Lancez Ollama avec la commande: ollama serve")

## 3. Initialiser le système RAG multimodal

Maintenant, initialisons notre système RAG multimodal :

In [ ]:
# Modèle Ollama à utiliser
LLM_MODEL = "qwen2.5:3b"

# Initialiser le système RAG
print("Initialisation du système RAG multimodal...")
rag_system = MultimodalRAG(llm_name=LLM_MODEL)
print("Système RAG multimodal initialisé avec succès!")

## 4. Explorer les embeddings CLIP

CLIP (Contrastive Language-Image Pre-training) est un modèle qui aligne les représentations de texte et d'image dans un même espace vectoriel. Voyons comment cela fonctionne :